<a href="https://colab.research.google.com/github/kash2023k/Pytorch-Project1/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Identifying 0-9 hand written numbers**

In [1]:
# Importing all necessary library for CNN
from keras.models import Sequential
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from keras.layers import Dense, Conv2D, Flatten

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline



In [2]:
# Convert MNIST image files into Tensor of 4 dimensions (# of images , height , width , color channels)

transform = transforms.ToTensor()

In [3]:
# Train Data

train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)

100%|██████████| 9.91M/9.91M [00:00<00:00, 43.2MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.08MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 10.0MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.33MB/s]


In [4]:
# Test Data

test_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transform)

In [5]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [7]:
# Lets create a batch size of images .. 10

train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False) # We dont want to Shuffle

In [8]:
# Define CNN Model
# Describe CNN layer - 2 layers

conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1)
conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1)


In [9]:
# Lets grab one MNIST Record/image

for i , (X_train, y_train) in enumerate(train_data):
    break


In [10]:
X_train.shape

torch.Size([1, 28, 28])

In [11]:
x = X_train.view(1,1,28,28) # You can use reshape also


In [12]:
# Perform out first convolution

x = F.relu(conv1(x)) #Rectified Linear Unit for our activation function

In [13]:
x.shape # 1 - Single Image , 6- filters , but dimension has reduced from 28x28 to 26x26 because the convolution got rid of the extra padding

torch.Size([1, 6, 26, 26])

In [14]:
# Pass through the pooling layer

x = F.max_pool2d(x, 2, 2) # Kernel Size of 2 and stride size of 2

In [15]:
x.shape # 26 /2 = 13 thus pooled 26x26 to 13x13

torch.Size([1, 6, 13, 13])

In [16]:
# Lets do our second convolutional layer

x= F.relu(conv2(x))

In [17]:
x.shape # lost two pixel on the outside for padding

torch.Size([1, 16, 11, 11])

In [18]:
# Pooling the 2nd convolutional layer

x = F.max_pool2d(x, 2, 2)

In [19]:
x.shape # 11/2 = 5.5 amd we cant rounup because we have lost the data already which cant be recovered and thus we pool dowm

torch.Size([1, 16, 5, 5])

In [20]:
# Model Class

class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1,6,3,1)
    self.conv2 = nn.Conv2d(6,16,3,1)

    # implementing fully connected layers
    self.fc1 = nn.Linear(5*5*16, 120) # 5*5*16 from the x shape after pooling and 120 is the number of neurons
    self.fc2 = nn.Linear(120, 84) # 120 neunrons from previous connected layer to 84 neurons (arbitrary) next
    self.fc3 = nn.Linear(84,10) # 84 from previous connected layer to 10 as we have 10 classes of number 0-9

  def forward(self, X):
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X, 2, 2) #2X2 kernal and a stride of 2

    # Second Pass
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X, 2, 2)

    #Re-view to flatten it out

    X = X.view(-1, 5*5*16) # Negative so that we can vary the batch size

    #Fully connected layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)
    return F.log_softmax(X, dim=1)


In [21]:
# Create an instance of our model
# Creating a seed

torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [22]:
# Loss function optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # The smaller rate is longer it will time to take

In [23]:
import time #Just curious how much time it takes

start_time = time.time()

#Create variable to track things
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

# For loop for epochs
for i in range(epochs):
  trn_corr = 0
  tst_corr = 0

  #Training
  for b, (X_train, y_train) in enumerate(train_loader):
    b+=1 #Start our bacth with 1
    y_pred = model(X_train)  # Prediction values from training set
    loss = criterion(y_pred, y_train)  # How far off are we from actual
    predicted = torch.max(y_pred.data, 1)[1] # Add up numbers for correct prediction
    batch_corr = (predicted == y_train).sum() # Sum all correct prediction from this batch. True = 1 and False =0 and we sum those off
    trn_corr += batch_corr # Keep a track as we go along in training

    # Update our parameters

    optimizer.zero_grad() # Reset the gradient
    loss.backward() # Back propagation
    optimizer.step() # Update the parameters

    #Print out some results

    if b%600 == 0:
      print(f'Epoch: {i}  Batch: {b}  Loss: {loss.item()}')


  # Append loss and correct

  train_losses.append(loss)
  train_correct.append(trn_corr)

  # Testing

  with torch.no_grad():
    for b, (X_test, y_test) in enumerate(test_loader):
      y_val = model(X_test)
      predicted = torch.max(y_val.data, 1)[1]
      tst_corr += (predicted == y_test).sum()

  loss = criterion(y_val, y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)

print(f'Epoch: {i}  Batch: {b}  Loss: {loss.item()}')



current = time.time()
total_time = current - start_time
print("The time taken was : ",total_time/60)

Epoch: 0  Batch: 600  Loss: 0.1623610556125641
Epoch: 0  Batch: 1200  Loss: 0.1502392590045929
Epoch: 0  Batch: 1800  Loss: 0.4744560718536377
Epoch: 0  Batch: 2400  Loss: 0.14238706231117249
Epoch: 0  Batch: 3000  Loss: 0.007758188061416149
Epoch: 0  Batch: 3600  Loss: 0.3836284875869751
Epoch: 0  Batch: 4200  Loss: 0.0038223876617848873
Epoch: 0  Batch: 4800  Loss: 0.0021286322735249996
Epoch: 0  Batch: 5400  Loss: 0.0569545142352581
Epoch: 0  Batch: 6000  Loss: 0.00038789428072050214
Epoch: 1  Batch: 600  Loss: 0.005851339548826218
Epoch: 1  Batch: 1200  Loss: 0.3855525553226471
Epoch: 1  Batch: 1800  Loss: 0.004819948226213455
Epoch: 1  Batch: 2400  Loss: 0.003216963727027178
Epoch: 1  Batch: 3000  Loss: 0.0332382395863533
Epoch: 1  Batch: 3600  Loss: 0.5372857451438904
Epoch: 1  Batch: 4200  Loss: 0.04561494290828705
Epoch: 1  Batch: 4800  Loss: 0.0007510822033509612
Epoch: 1  Batch: 5400  Loss: 0.0001173773780465126
Epoch: 1  Batch: 6000  Loss: 0.14201366901397705
Epoch: 2  Batch